# Week 11 — Storytelling & Executive Summary (스토리텔링/요약)

이 노트북은 **실전 통계 → 해석 → 비즈니스 액션**을 훈련하기 위한 템플릿입니다.
각 섹션의 ✅ 체크포인트를 채우면서 진행하세요.


## 1) Business Questions
- 분석 결과를 '숫자'가 아니라 '결정'으로 연결할 수 있는가?
- 한 페이지 요약으로 상사/비즈니스팀을 설득할 수 있는가?

## 2) Why this matters (Theory)
- 좋은 분석은 '질문 → 근거 → 결정' 구조
- 효과 크기/CI/가드레일을 포함하면 신뢰가 올라감
- 차트는 3개면 충분: (1)현상, (2)원인, (3)액션

## 3) Setup & Data Load

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf

DATA_PATH = "data/sales_marketing_example_2025.csv"
df = pd.read_csv(DATA_PATH, parse_dates=["date"])

# Quick sanity checks
display(df.head())
print("rows, cols:", df.shape)
print("missing rate (top):")
display(df.isna().mean().sort_values(ascending=False).head(10))

## 4) Helper Functions

In [ ]:
def one_sentence_insight(prompt: str):
    """Write your one-sentence business insight below."""
    print("\n🧾 One-sentence insight (fill this):")
    print(" -", prompt)

def bootstrap_mean_diff(x, y, iters=5000, seed=0):
    """Bootstrap CI for difference in means (x - y)."""
    r = np.random.default_rng(seed)
    x = np.asarray(pd.Series(x).fillna(0))
    y = np.asarray(pd.Series(y).fillna(0))
    diffs = []
    for _ in range(iters):
        xs = r.choice(x, size=len(x), replace=True)
        ys = r.choice(y, size=len(y), replace=True)
        diffs.append(xs.mean() - ys.mean())
    diffs = np.array(diffs)
    return diffs.mean(), np.quantile(diffs, [0.025, 0.975])

def rate_ci_wilson(successes, n, alpha=0.05):
    """Wilson score interval for a proportion."""
    if n == 0:
        return (np.nan, np.nan)
    z = stats.norm.ppf(1 - alpha/2)
    phat = successes / n
    denom = 1 + z**2/n
    center = (phat + z**2/(2*n)) / denom
    half = z * np.sqrt((phat*(1-phat) + z**2/(4*n)) / n) / denom
    return center - half, center + half

---
## 5) Analysis

### 5.1 Build your 3-chart pack (skeleton)
✅ 아래 3개 차트를 완성해 요약에 붙이세요.

In [ ]:
# Chart 1: Monthly revenue
tmp = df.copy()
tmp['month'] = tmp['date'].dt.to_period('M').dt.to_timestamp()
monthly = tmp.assign(revenue=tmp['revenue'].fillna(0)).groupby('month')['revenue'].sum()

plt.figure()
monthly.plot(title="Chart 1 — Monthly Revenue")
plt.show()

# Chart 2: Conversion by campaign
cr = df.groupby('campaign')['conversions'].mean().sort_values(ascending=False)
plt.figure()
cr.plot(kind='bar', title="Chart 2 — Conversion rate by campaign")
plt.show()

# Chart 3: Margin vs discount (converted only)
m = df[df['revenue'].fillna(0) > 0].copy()
m['margin_rate'] = m['gross_margin']/m['revenue']
plt.figure()
plt.scatter(m['discount_rate'], m['margin_rate'], s=6)
plt.title("Chart 3 — Margin rate vs Discount rate")
plt.xlabel("discount_rate")
plt.ylabel("margin_rate")
plt.show()

In [ ]:
one_sentence_insight('이 3개 차트로 어떤 스토리를 만들지 한 문장')

### 5.2 Executive summary fill-in
✅ 아래를 채우면 바로 메일/슬라이드에 붙여넣을 수 있습니다.

In [ ]:
print("""- 핵심 질문:
- 결론(한 문장):
- 근거 지표:
- 비즈니스 임팩트:
- 권장 액션:
- 추가 분석 필요:""")

In [ ]:
one_sentence_insight('결론(한 문장)을 강하게(액션까지) 쓰기')

---
## 6) Interpretation Checklist
- 결과가 '유의'하면, **효과 크기/CI**가 실무적으로 의미 있는가?
- 결과가 '비유의'하면, **표본 부족/측정오류/세그먼트 혼합** 가능성은?
- 액션을 한다면, **누구에게/어디에/얼마나** 적용할 것인가?



## Executive Summary (메일/슬라이드에 그대로 붙여넣기)

- **핵심 질문:**  
- **결론(한 문장):**  
- **근거 지표:** (효과크기 / p-value / CI 등)  
- **비즈니스 임팩트:** (예: 월 매출 +X, 마진 -Y 위험)  
- **권장 액션:** (실험/타겟 변경/채널 믹스 조정)  
- **추가 분석 필요:**  
